In [17]:
import numpy as np
import torch
import torch.optim as optim
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tensorboardX import SummaryWriter
from sklearn import metrics
import tensorboard

### Модель

In [18]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self._model = nn.Sequential(
            nn.Conv2d(1, 32, 5, padding=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),
            nn.Flatten(),
            nn.Linear(64 * 13 * 13, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10),
            nn.LogSoftmax(dim=1),
        )

    def forward(self, x):
        return self._model(x)

#### Функция обучения

In [19]:
def test(model, test_dataloader):
    model.eval()
    test_loss = 0
    test_predictions = []
    test_targets = []

    with torch.no_grad():
        for data, target in test_dataloader:
            data, target = Variable(data), Variable(target)
            output = model(data)

            test_loss += F.nll_loss(output, target, reduction="sum").item()
            test_predictions.append(output.data.max(1)[1])
            test_targets.append(target)

    test_predictions = torch.cat(test_predictions).cpu()
    test_targets = torch.cat(test_targets).cpu()

    test_loss /= len(test_dataloader.dataset)

    test_accuracy = metrics.accuracy_score(test_targets, test_predictions)
    test_precision = metrics.precision_score(test_targets, test_predictions, average="macro", zero_division=0)

    return test_loss, test_accuracy, test_precision


def train(model, train_dataloader, test_dataloader, epochs=5):
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    iteration, log_interval = 1, 10_000 // batch_size

    train_losses, test_losses, train_accuracies, test_accuracies, test_precisions = [], [], [], [], []

    for epoch in range(1, epochs + 1):
        print(f"\nEpoch {epoch}")
        print("-------------------------------")
        model.train()
        train_loss_epoch = []
        train_predictions = []
        train_targets = []
        for batch, (data, target) in enumerate(train_dataloader):
            # Train single batch
            data, target = Variable(data), Variable(target)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            train_loss_epoch.append(loss.data.item())
            optimizer.step()
            train_predictions.append(output.data.max(1)[1])
            train_targets.append(target)

            if iteration % log_interval == 0:
                test_loss, test_accuracy, test_precision = test(model, test_dataloader)
                model.train()
                train_losses.append(loss.data.item())
                l = np.mean(train_loss_epoch)
                train_loss_epoch = []
                test_losses.append(test_loss)
                test_accuracies.append(test_accuracy)
                test_precisions.append(test_precision)

                train_predictions = torch.cat(train_predictions).cpu()
                train_targets = torch.cat(train_targets).cpu()
                acc = metrics.accuracy_score(train_targets, train_predictions)
                train_predictions, train_targets = [], []
                train_accuracies.append(acc)
                print(
                    f"{(batch + 1) * len(data):>5d}/{len(train_dataloader.dataset):>5d} | train_loss: {l:>5f} | train accuracy: {acc:>4f} | accuracy: {test_accuracy:>4f}")

            iteration += 1

    return train_losses, test_losses, test_accuracies, test_precisions, train_accuracies

#### Создаем тренировочный и тестовый датасеты

In [20]:
mnist_training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

mnist_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [21]:
epochs, batch_size = 6, 100
mnist_train_dataloader = DataLoader(mnist_training_data, batch_size=batch_size, shuffle=True)
mnist_test_dataloader = DataLoader(mnist_test_data, batch_size=batch_size)

#### Обучение Mnist

In [22]:
model_mnist = MnistModel()
train_losses, test_losses, test_accuracies, test_precisions, train_accuracies = train(
    model_mnist,
    mnist_train_dataloader,
    mnist_test_dataloader,
    epochs
)


Epoch 1
-------------------------------
10000/60000 | train_loss: 0.762698 | train accuracy: 0.754800 | accuracy: 0.951900
20000/60000 | train_loss: 0.292527 | train accuracy: 0.910000 | accuracy: 0.958600
30000/60000 | train_loss: 0.250663 | train accuracy: 0.926600 | accuracy: 0.971500
40000/60000 | train_loss: 0.215386 | train accuracy: 0.935500 | accuracy: 0.966900
50000/60000 | train_loss: 0.203999 | train accuracy: 0.940900 | accuracy: 0.968400
60000/60000 | train_loss: 0.209140 | train accuracy: 0.940500 | accuracy: 0.973500

Epoch 2
-------------------------------
10000/60000 | train_loss: 0.187973 | train accuracy: 0.942900 | accuracy: 0.978700
20000/60000 | train_loss: 0.178187 | train accuracy: 0.947000 | accuracy: 0.974700
30000/60000 | train_loss: 0.174164 | train accuracy: 0.948500 | accuracy: 0.980200
40000/60000 | train_loss: 0.173066 | train accuracy: 0.949700 | accuracy: 0.978400
50000/60000 | train_loss: 0.175725 | train accuracy: 0.949500 | accuracy: 0.972700
60000

In [23]:
torch.save(model_mnist.state_dict(), "models/mnist.torch")

In [24]:
writer = SummaryWriter("runs/mnist")
for i in range(len(test_losses)):
    writer.add_scalar("Train loss", train_losses[i], i)
    writer.add_scalar("Test loss", test_losses[i], i)
    writer.add_scalar("Test accuracy", test_accuracies[i], i)
    writer.add_scalar("Test precisions", test_precisions[i], i)
    writer.add_scalar("Train accuracy", train_accuracies[i], i)
writer.close()

#### Создаем тренировочный и тестовый датасеты

In [25]:
fashion_training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

fashion_test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [26]:
epochs, batch_size = 6, 100
fashion_train_dataloader = DataLoader(fashion_training_data, batch_size=batch_size, shuffle=True)
fashion_test_dataloader = DataLoader(fashion_test_data, batch_size=batch_size)

#### Обучение Fashion MNIST

In [27]:
model_fashion = MnistModel()
train_losses, test_losses, test_accuracies, test_precisions, train_accuracies = train(
    model_fashion,
    fashion_train_dataloader,
    fashion_test_dataloader,
    epochs
)


Epoch 1
-------------------------------
10000/60000 | train_loss: 0.856991 | train accuracy: 0.704800 | accuracy: 0.823700
20000/60000 | train_loss: 0.545447 | train accuracy: 0.809500 | accuracy: 0.854600
30000/60000 | train_loss: 0.474532 | train accuracy: 0.831200 | accuracy: 0.856500
40000/60000 | train_loss: 0.447456 | train accuracy: 0.836300 | accuracy: 0.866300
50000/60000 | train_loss: 0.431988 | train accuracy: 0.848100 | accuracy: 0.857400
60000/60000 | train_loss: 0.417572 | train accuracy: 0.850100 | accuracy: 0.874100

Epoch 2
-------------------------------
10000/60000 | train_loss: 0.390538 | train accuracy: 0.856100 | accuracy: 0.863700
20000/60000 | train_loss: 0.393894 | train accuracy: 0.855600 | accuracy: 0.857800
30000/60000 | train_loss: 0.404803 | train accuracy: 0.849300 | accuracy: 0.875100
40000/60000 | train_loss: 0.391039 | train accuracy: 0.860300 | accuracy: 0.886300
50000/60000 | train_loss: 0.394838 | train accuracy: 0.856300 | accuracy: 0.871600
60000

In [28]:
writer = SummaryWriter("runs/fashion")
for i in range(len(test_losses)):
    writer.add_scalar("Train loss", train_losses[i], i)
    writer.add_scalar("Test loss", test_losses[i], i)
    writer.add_scalar("Test accuracy", test_accuracies[i], i)
    writer.add_scalar("Test precisions", test_precisions[i], i)
    writer.add_scalar("Train accuracy", train_accuracies[i], i)
writer.close()

#### Дообучение Fashion MNIST с помощью MNIST

In [29]:
fashion_continue_model = MnistModel()
fashion_continue_model.load_state_dict(torch.load("models/mnist.torch"))
train_losses, test_losses, test_accuracies, test_precisions, train_accuracies = train(
    fashion_continue_model,
    fashion_train_dataloader,
    fashion_test_dataloader,
    epochs
)


Epoch 1
-------------------------------
10000/60000 | train_loss: 1.555941 | train accuracy: 0.492000 | accuracy: 0.745300
20000/60000 | train_loss: 0.792950 | train accuracy: 0.698300 | accuracy: 0.790300
30000/60000 | train_loss: 0.684770 | train accuracy: 0.739200 | accuracy: 0.800500
40000/60000 | train_loss: 0.625312 | train accuracy: 0.759600 | accuracy: 0.818500
50000/60000 | train_loss: 0.576647 | train accuracy: 0.777300 | accuracy: 0.833300
60000/60000 | train_loss: 0.554149 | train accuracy: 0.791600 | accuracy: 0.839200

Epoch 2
-------------------------------
10000/60000 | train_loss: 0.544152 | train accuracy: 0.794000 | accuracy: 0.838000
20000/60000 | train_loss: 0.532387 | train accuracy: 0.803700 | accuracy: 0.842800
30000/60000 | train_loss: 0.503077 | train accuracy: 0.811100 | accuracy: 0.845000
40000/60000 | train_loss: 0.512161 | train accuracy: 0.806900 | accuracy: 0.854000
50000/60000 | train_loss: 0.505193 | train accuracy: 0.808500 | accuracy: 0.833300
60000

In [30]:
writer = SummaryWriter("runs/fashion_continue")
for i in range(len(test_losses)):
    writer.add_scalar("Train loss", train_losses[i], i)
    writer.add_scalar("Test loss", test_losses[i], i)
    writer.add_scalar("Test accuracy", test_accuracies[i], i)
    writer.add_scalar("Test precisions", test_precisions[i], i)
    writer.add_scalar("Train accuracy", train_accuracies[i], i)
writer.close()

#### Дообучение Fashion MNIST с помощью MNIST с заморозкой свёрточных слоёв

In [31]:
fashion_freeze_model = MnistModel()
fashion_freeze_model.load_state_dict(torch.load("models/mnist.torch"))

<All keys matched successfully>

In [32]:
for name, param in fashion_freeze_model.named_parameters():
    print(f"{name:<20s} {param.requires_grad}")

_model.0.weight      True
_model.0.bias        True
_model.2.weight      True
_model.2.bias        True
_model.7.weight      True
_model.7.bias        True
_model.10.weight     True
_model.10.bias       True


In [33]:
freeze_params = [
    "_model.0.weight",
    "_model.0.bias",
    "_model.2.weight",
    "_model.2.bias",
]

In [34]:
for name, param in fashion_freeze_model.named_parameters():
    if name in freeze_params:
        param.requires_grad = False

In [35]:
for name, param in fashion_freeze_model.named_parameters():
    print(f"{name:<20s} {param.requires_grad}")

_model.0.weight      False
_model.0.bias        False
_model.2.weight      False
_model.2.bias        False
_model.7.weight      True
_model.7.bias        True
_model.10.weight     True
_model.10.bias       True


In [36]:
train_losses, test_losses, test_accuracies, test_precisions, train_accuracies = train(
    fashion_freeze_model,
    fashion_train_dataloader,
    fashion_test_dataloader,
    epochs // 2
)


Epoch 1
-------------------------------
10000/60000 | train_loss: 1.644562 | train accuracy: 0.481200 | accuracy: 0.717800
20000/60000 | train_loss: 0.952638 | train accuracy: 0.640800 | accuracy: 0.754500
30000/60000 | train_loss: 0.820286 | train accuracy: 0.684600 | accuracy: 0.731200
40000/60000 | train_loss: 0.787053 | train accuracy: 0.701200 | accuracy: 0.774100
50000/60000 | train_loss: 0.755027 | train accuracy: 0.707800 | accuracy: 0.784600
60000/60000 | train_loss: 0.719157 | train accuracy: 0.722900 | accuracy: 0.793500

Epoch 2
-------------------------------
10000/60000 | train_loss: 0.698781 | train accuracy: 0.729100 | accuracy: 0.799100
20000/60000 | train_loss: 0.699909 | train accuracy: 0.732000 | accuracy: 0.800600
30000/60000 | train_loss: 0.658780 | train accuracy: 0.749800 | accuracy: 0.782700
40000/60000 | train_loss: 0.675029 | train accuracy: 0.742000 | accuracy: 0.798100
50000/60000 | train_loss: 0.658215 | train accuracy: 0.753200 | accuracy: 0.809700
60000

In [37]:
writer = SummaryWriter("runs/fashion_freeze")
for i in range(len(test_losses)):
    writer.add_scalar("Train loss", train_losses[i], i)
    writer.add_scalar("Test loss", test_losses[i], i)
    writer.add_scalar("Test accuracy", test_accuracies[i], i)
    writer.add_scalar("Test precisions", test_precisions[i], i)
    writer.add_scalar("Train accuracy", train_accuracies[i], i)
writer.close()

#### Дообучение Fashion MNIST с помощью MNIST с разморозкой свёрточных слоёв

In [38]:
for name, param in fashion_freeze_model.named_parameters():
    if name in freeze_params:
        param.requires_grad = True

In [39]:
for name, param in fashion_freeze_model.named_parameters():
    print(f"{name:<20s} {param.requires_grad}")

_model.0.weight      True
_model.0.bias        True
_model.2.weight      True
_model.2.bias        True
_model.7.weight      True
_model.7.bias        True
_model.10.weight     True
_model.10.bias       True


In [40]:
train_losses_2, test_losses_2, test_accuracies_2, test_precisions_2, train_accuracies_2 = train(
    fashion_freeze_model,
    fashion_train_dataloader,
    fashion_test_dataloader,
    epochs // 2
)


Epoch 1
-------------------------------
10000/60000 | train_loss: 0.626990 | train accuracy: 0.768100 | accuracy: 0.828200
20000/60000 | train_loss: 0.587343 | train accuracy: 0.781700 | accuracy: 0.830400
30000/60000 | train_loss: 0.567347 | train accuracy: 0.784400 | accuracy: 0.834800
40000/60000 | train_loss: 0.549700 | train accuracy: 0.788100 | accuracy: 0.835400
50000/60000 | train_loss: 0.574835 | train accuracy: 0.779900 | accuracy: 0.826900
60000/60000 | train_loss: 0.557465 | train accuracy: 0.790300 | accuracy: 0.838800

Epoch 2
-------------------------------
10000/60000 | train_loss: 0.537068 | train accuracy: 0.795500 | accuracy: 0.849800
20000/60000 | train_loss: 0.547438 | train accuracy: 0.792200 | accuracy: 0.837200
30000/60000 | train_loss: 0.510699 | train accuracy: 0.808200 | accuracy: 0.850600
40000/60000 | train_loss: 0.492890 | train accuracy: 0.817000 | accuracy: 0.854500
50000/60000 | train_loss: 0.503728 | train accuracy: 0.814200 | accuracy: 0.845700
60000

In [41]:
train_losses.extend(train_losses_2)
test_losses.extend(test_losses_2)
test_accuracies.extend(test_accuracies_2)
test_precisions.extend(test_precisions_2)
train_accuracies.extend(train_accuracies_2)

In [42]:
writer = SummaryWriter("runs/fashion_freeze_unfreeze")
for i in range(len(test_losses)):
    writer.add_scalar("Train loss", train_losses[i], i)
    writer.add_scalar("Test loss", test_losses[i], i)
    writer.add_scalar("Test accuracy", test_accuracies[i], i)
    writer.add_scalar("Test precisions", test_precisions[i], i)
    writer.add_scalar("Train accuracy", train_accuracies[i], i)
writer.close()

In [2]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [1]:
%tensorboard --logdir runs/mnist --port 20000
%tensorboard --logdir runs/fashion --port 20001
%tensorboard --logdir runs/fashion_continue --port 20002
%tensorboard --logdir runs/fashion_freeze --port 20003
%tensorboard --logdir runs/fashion_freeze_unfreeze --port 20004

Launching TensorBoard...

Launching TensorBoard...

Launching TensorBoard...

Launching TensorBoard...

Launching TensorBoard...